# Evaluation of tsdr on localization

In [1]:
%reload_ext autoreload
%autoreload 2

In [21]:
import numpy as np
import pandas as pd
import scipy.stats
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = "DejaVu Sans"
plt.rcParams["font.size"] = 7
plt.rcParams['xtick.labelsize'] = 9
plt.rcParams['ytick.labelsize'] = 9
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['axes.linewidth'] = 1.0
plt.rcParams['axes.grid'] = True

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore', FutureWarning)

import sys
sys.path.append("../")

import logging
logger = logging.getLogger()
if not logger.handlers:
    handler = logging.StreamHandler(sys.stdout)
    handler.setLevel(logging.INFO)
    logger.addHandler(handler)
    logger.setLevel(logging.INFO)

In [4]:
SOCKSHOP_DATASET_ID = "9n6mf"

from meltria.priorknowledge.priorknowledge import SockShopKnowledge

pk = SockShopKnowledge(
    target_metric_types={
        "containers": True,
        "services": True,
        "middlewares": True,
        "nodes": False,
    },
    mappings={"nodes-containers": {}},
)

In [25]:
from notebooklib import localization


results = localization.grid_dataset({
    "phase2only_hierarchy-single_sbd_medoid_only_ctnrs",
    "phase1only_kstest_only_ctnrs",
    "phase1only_ediagnosis_only_ctnrs",
    "phase1only_fluxinfer_only_ctnrs",
}, SOCKSHOP_DATASET_ID, n=10, diag_options=dict(
    enable_prior_knowledge=False,
    pc_library="cdt",
    cg_algo="pc",
    pc_citest_alpha=0.05,
    pc_citest="gaussian",
    pc_variant="stable",
    disable_orientation=False,
    disable_ci_edge_cut=False,
    walk_method="monitorrank",
    root_metric_type="latency",
))
for cache_suffix, score_df in results:
    display(cache_suffix, score_df)

INFO:root:Loading 9n6mf with phase1only_kstest_only_ctnrs...


Loading 9n6mf with phase1only_kstest_only_ctnrs...


INFO:root:Processing 9n6mf with phase1only_kstest_only_ctnrs...


Processing 9n6mf with phase1only_kstest_only_ctnrs...


No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.
No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.
No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.
No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.
No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.
No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.
No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.
No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.
No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.
No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.
No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.

...

,#cases (metric),AC@K (metric),AVG@K (metric),#cases (container),AC@K (container),AVG@K (container),#cases (service),AC@K (service),AVG@K (service)
1,27,0.518519,0.518519,105,0.142857,0.142857,105,0.047619,0.047619
2,27,0.629630,0.574074,105,0.219048,0.180952,105,0.247619,0.147619
3,27,0.777778,0.641975,105,0.342857,0.234921,105,0.447619,0.247619
4,27,0.851852,0.694444,105,0.409524,0.278571,105,0.590476,0.333333
5,27,0.851852,0.725926,105,0.409524,0.304762,105,0.733333,0.413333
6,27,0.851852,0.746914,105,0.409524,0.322222,105,0.761905,0.471429
7,27,0.888889,0.767196,105,0.409524,0.334694,105,0.800000,0.518367
8,27,0.962963,0.791667,105,0.409524,0.344048,105,0.800000,0.553571
9,27,1.000000,0.814815,105,0.409524,0.351323,105,0.800000,0.580952
10,27,1.000000,0.833333,105,0.409524,0.357143,105,0.800000,0.602857


'phase1only_ediagnosis_only_ctnrs'

,#cases (metric),AC@K (metric),AVG@K (metric),#cases (container),AC@K (container),AVG@K (container),#cases (service),AC@K (service),AVG@K (service)
1,27,0.444444,0.444444,105,0.104762,0.104762,105,0.066667,0.066667
2,27,0.592593,0.518519,105,0.257143,0.180952,105,0.247619,0.157143
3,27,0.740741,0.592593,105,0.352381,0.238095,105,0.514286,0.276190
4,27,0.814815,0.648148,105,0.428571,0.285714,105,0.657143,0.371429
5,27,0.925926,0.703704,105,0.466667,0.321905,105,0.733333,0.443810
6,27,0.962963,0.746914,105,0.466667,0.346032,105,0.790476,0.501587
7,27,1.000000,0.783069,105,0.476190,0.364626,105,0.828571,0.548299
8,27,1.000000,0.810185,105,0.476190,0.378571,105,0.828571,0.583333
9,27,1.000000,0.831276,105,0.476190,0.389418,105,0.828571,0.610582
10,27,1.000000,0.848148,105,0.476190,0.398095,105,0.828571,0.632381


'phase1only_fluxinfer_only_ctnrs'

,#cases (metric),AC@K (metric),AVG@K (metric),#cases (container),AC@K (container),AVG@K (container),#cases (service),AC@K (service),AVG@K (service)
1,28,0.357143,0.357143,105,0.161905,0.161905,105,0.038095,0.038095
2,28,0.500000,0.428571,105,0.285714,0.223810,105,0.247619,0.142857
3,28,0.714286,0.523810,105,0.361905,0.269841,105,0.438095,0.241270
4,28,0.821429,0.598214,105,0.400000,0.302381,105,0.619048,0.335714
5,28,0.892857,0.657143,105,0.419048,0.325714,105,0.733333,0.415238
6,28,0.964286,0.708333,105,0.438095,0.344444,105,0.809524,0.480952
7,28,1.000000,0.750000,105,0.438095,0.357823,105,0.838095,0.531973
8,28,1.000000,0.781250,105,0.447619,0.369048,105,0.838095,0.570238
9,28,1.000000,0.805556,105,0.447619,0.377778,105,0.838095,0.600000
10,28,1.000000,0.825000,105,0.447619,0.384762,105,0.838095,0.623810


In [27]:
from notebooklib import localization

results = localization.grid_dataset([
    "kstest_hierarchy-single_sbd_medoid_only_ctnrs",
    "ediagnosis_hierarchy-single_sbd_medoid_only_ctnrs",
    "fluxinfer_hierarchy-single_sbd_medoid_only_ctnrs",
], SOCKSHOP_DATASET_ID, n=10, diag_options=dict(
    enable_prior_knowledge=False,
    pc_library="cdt",
    cg_algo="pc",
    pc_citest_alpha=0.05,
    pc_citest="gaussian",
    pc_variant="stable",
    disable_orientation=False,
    disable_ci_edge_cut=False,
    walk_method="monitorrank",
    root_metric_type="latency",
))
for cache_suffix, score_df in results:
    display(cache_suffix, score_df)

INFO:root:Loading 9n6mf with kstest_hierarchy-single_sbd_medoid_only_ctnrs...


Loading 9n6mf with kstest_hierarchy-single_sbd_medoid_only_ctnrs...
